In [1]:
import json
from datetime import datetime 
import pandas as pd
import csv
import requests

In [2]:
url = 'https://api.github.com/repos/tensorflow/datasets' # url of repo to be analysed
info = requests.get(url)
if(info.ok):
    startyear = datetime.strptime(json.loads(info.text)['created_at'], "%Y-%m-%dT%H:%M:%SZ").year
    endyear = datetime.strptime(json.loads(info.text)['updated_at'], "%Y-%m-%dT%H:%M:%SZ").year
else : 
    info.raise_for_status()

In [3]:
indexes = []
for year in range(startyear, endyear+1):
    for quarter in ['Q1', 'Q2', 'Q3', 'Q4']:
        indexes.append(str(year) + '_' + quarter)
        
# dictionary mapping month to quarter
quarter = {1:'Q1', 2:'Q1', 3:'Q1', 4:'Q2', 5:'Q2', 6:'Q2', 7:'Q3', 8:'Q3', 9:'Q3', 10:'Q4', 11:'Q4', 12:'Q4'}

In [26]:
with open("./tf_analysis.json") as file:
    for line in file:
        line = json.loads(line)
        if(line['category']=='commit'):
            commits_(line['data'])
        elif(line['category']=='pull_request'):
            pr_(line['data'])
        else:
            issues_(line['data'])

In [21]:
#creating the issue_df dataframe
issue_df = pd.DataFrame(columns = ['index', '#issues', 'users', '#new_users'])
issue_df['index'] = indexes
issue_df['users'] = [[] for _ in range(len(indexes))]
issue_df['#new_users'] = 0
issue_df['#issues'] = 0

# function to populate the issue_df dataframe (leaving the #new_users column for now)
def issues_(issue_data):
    created = datetime.strptime(issue_data['created_at'], "%Y-%m-%dT%H:%M:%SZ")
    index = str(created.year) + "_" + quarter[created.month]
    issue_df.loc[issue_df['index']==index, 'users'].apply(lambda x: x.append(issue_data['user_data']['login']))
    issue_df.loc[issue_df['index']==index, '#issues'] = issue_df.loc[issue_df['index']==index, '#issues'].apply(lambda x : x+1)

In [22]:
#creating the pr_df dataframe
pr_df = pd.DataFrame(columns = ['index', '#prs', 'users', '#new_users'])
pr_df['index'] = indexes
pr_df['users'] = [[] for _ in range(len(indexes))]
pr_df['#new_users'] = 0
pr_df['#prs'] = 0

# function to populate the pr_df dataframe (leaving the #new_users column for now)
def pr_(pr_data):
    created = datetime.strptime(pr_data['created_at'], "%Y-%m-%dT%H:%M:%SZ")
    index = str(created.year) + "_" + quarter[created.month]
    pr_df.loc[pr_df['index']==index, 'users'].apply(lambda x: x.append(pr_data['user_data']['login']))
    pr_df.loc[pr_df['index']==index, '#prs'] = pr_df.loc[pr_df['index']==index, '#prs'].apply(lambda x : x+1)

In [23]:
commit_df = pd.DataFrame(columns = ['index', '#commits', 'commiters', '#new_commiters'])
commit_df['index'] = indexes
commit_df['commiters'] = [list() for _ in range(len(indexes))]
commit_df['#new_commiters'] = 0
commit_df['#commits'] = 0

# function to populate the commit_df dataframe (leaving the #new_commiters column for now)
def commits_(cdata):
    commit_date = datetime.strptime(cdata['CommitDate'],"%a %b %d %H:%M:%S %Y %z")
    index = str(commit_date.year) + "_" + quarter[commit_date.month]
    commit_df.loc[commit_df['index']==index, 'commiters'].apply(lambda x : x.append(cdata['Author']))
    commit_df.loc[commit_df['index']==index, '#commits'] = commit_df.loc[commit_df['index']==index, '#commits'].apply(
                                                                                         lambda x :  x+1)

In [27]:
commit_df.head(8)

,index,#commits,commiters,#new_commiters
0,2018_Q1,0,[],0
1,2018_Q2,0,[],0
2,2018_Q3,10,"[Ryan Sepassi <rsepassi@google.com>, Ryan Sepa...",0
3,2018_Q4,272,"[Ryan Sepassi <rsepassi@google.com>, Ryan Sepa...",0
4,2019_Q1,414,"[Marcin Michalski <michalski@google.com>, Marc...",0
5,2019_Q2,0,[],0
6,2019_Q3,0,[],0
7,2019_Q4,0,[],0


In [28]:
commit_df.loc[0, "#new_commiters"] = len(commit_df.loc[0,"commiters"])
pr_df.loc[0, "#new_users"] = len(pr_df.loc[0,"users"])
issue_df.loc[0, "#new_users"] = len(issue_df.loc[0,"users"])

In [29]:
commiters_uptil = set(commit_df.loc[0,'commiters'])
for i in range(1, len(indexes)):
    current_commiters = set(commit_df.loc[i,'commiters'])
    commit_df.loc[i,'#new_commiters'] = len(current_commiters.difference(commiters_uptil))
    commiters_uptil.update(current_commiters)
    
issue_contrib_uptil = set(issue_df.loc[0,'users'])
for i in range(1, len(indexes)):
    current_contributors = set(issue_df.loc[i,'users'])
    issue_df.loc[i,'#new_users'] = len(current_contributors.difference(issue_contrib_uptil))
    issue_contrib_uptil.update(current_contributors)
    
pr_users_uptil = set(pr_df.loc[0,'users'])
for i in range(1, len(indexes)):
    current_users = set(pr_df.loc[i,'users'])
    pr_df.loc[i,'#new_users'] = len(current_users.difference(pr_users_uptil))
    pr_users_uptil.update(current_users)

In [30]:
commit_df.head()

,index,#commits,commiters,#new_commiters
0,2018_Q1,0,[],0
1,2018_Q2,0,[],0
2,2018_Q3,10,"[Ryan Sepassi <rsepassi@google.com>, Ryan Sepa...",3
3,2018_Q4,272,"[Ryan Sepassi <rsepassi@google.com>, Ryan Sepa...",9
4,2019_Q1,414,"[Marcin Michalski <michalski@google.com>, Marc...",30


In [48]:
type(commit_date.year)

int

In [49]:
type(commit_date.month)

int